In [18]:
import os
from sklearn.decomposition import PCA
import random
from numpy.random import seed

seed(1)
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, Activation, BatchNormalization, Flatten
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.preprocessing import label_binarize
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

from tensorflow.keras.callbacks import EarlyStopping

In [19]:
vector_size = 3390
event_num = 2
droprate = 0.3

In [20]:
def DNN():
    print("________DNN_________")
    train_input = Input(shape=(vector_size,), name='Inputlayer')
    train_in = Dense(512, activation='relu')(train_input)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)

    train_in = Dense(256, activation='relu')(train_in)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)

    train_in = Dense(event_num)(train_in)
    out = Activation('softmax')(train_in)

    model = Model(train_input, out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

In [21]:
def readdata():
    path = 'pcaTrue.txt'
    path1 = 'pcaFalse.txt'

    all_matrix = []
    all_labels = []

    fp = open(path, 'r')
    fp_false = open(path1, 'r')
    lines = fp.readlines()
    for line in lines:
        line = line.split(" ")
        truedata = []
        for i in range(0, len(line)):
            truedata.append(float(line[i]))
        all_matrix.append(truedata)
        all_labels.append(0)

    lines2 = fp_false.readlines()
    for line in lines2:
        line = line.split(" ")
        falsedata = []
        for i in range(0, len(line)):
            falsedata.append(float(line[i]))
        all_matrix.append(falsedata)
        all_labels.append(1)
    cc = list(zip(all_matrix, all_labels))
    random.shuffle(cc)
    all_matrix[:], all_labels[:] = zip(*cc)
    return all_matrix, all_labels

In [22]:
def evaluate(pred_type, pred_score, y_test, event_num):
    y_one_hot = label_binarize(y_test, np.arange(event_num + 1))
    y_one_hot = y_one_hot[:, [0, 1]]

    result_auc_micro = roc_auc_score(y_one_hot, pred_score, average='micro')
    result_auc_macro = roc_auc_score(y_one_hot, pred_score, average='macro')
    return result_auc_micro, result_auc_macro

In [23]:
def get_index(label_matrix, event_num, seed, CV):
    index_all_class = np.zeros(len(label_matrix))
    for j in range(event_num):
        index = np.where(label_matrix == j)
        kf = KFold(n_splits=CV, shuffle=True, random_state=seed)
        k_num = 0
        for train_index, test_index in kf.split(range(len(index[0]))):
            index_all_class[index[0][test_index]] = k_num
            k_num += 1
    return index_all_class

In [24]:
def cross_validation(feature_matrix, label_matrix, clf_type, event_num, seed, CV):
    y_true = np.array([])
    y_pred = np.array([])
    y_score = np.zeros((0, event_num), dtype=float)
    label_matrix = np.array(label_matrix)
    feature_matrix = np.array(feature_matrix)
    index_all_class = get_index(label_matrix, event_num, seed, CV)

    matrix = []
    print("_____cross_validation_____")

    for k in range(CV):
        train_index = np.where(index_all_class != k)
        test_index = np.where(index_all_class == k)
        pred = np.zeros((len(test_index[0]), event_num), dtype=float)

        x_train = feature_matrix[train_index]
        x_test = feature_matrix[test_index]
        y_train = label_matrix[train_index]
        y_test = label_matrix[test_index]

        y_train_one_hot = np.array(y_train)
        y_train_one_hot = (np.arange(y_train_one_hot.max() + 1) == y_train[:, None]).astype(dtype='float32')

        y_test_one_hot = np.array(y_test)
        y_test_one_hot = (np.arange(y_test_one_hot.max() + 1) == y_test[:, None]).astype(dtype='float32')

        if clf_type == 'DNN':
            dnn = DNN()
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
            dnn.fit(x_train, y_train_one_hot, batch_size=64, epochs=100, validation_data=(x_test, y_test_one_hot),
                    callbacks=[early_stopping])

            pred += dnn.predict(x_test)
        else:
            print("_______ERROR___________")
        pred_score = pred / 1
        pred_type = np.argmax(pred_score, axis=1)
        y_true = np.hstack((y_true, y_test))
        y_pred = np.hstack((y_pred, pred_type))
        y_score = np.row_stack((y_score, pred_score))

        wfp = open(str(k) + '.txt', 'w')
        for i in range(len(y_test)):
            res = str(pred_score[i][0]) + ' ' + str(pred_score[i][1]) + ' ' + str(y_test[i]) + '\n'
            wfp.write(res)
        wfp.close()

        #########evaluate auc###########
        result_micro, result_macro = evaluate(pred_type, pred_score, y_test, event_num)
        print("idx, auc_micro, auc_macro: ", k, result_micro, result_macro)
    result_all_micro, result_all_macro = evaluate(y_pred, y_score, y_true, event_num)
    print("auc_micro_all, auc_macro_all: ", result_all_micro, result_all_macro)

In [25]:
def main():
    seed = 0
    CV = 10
    all_matrix, all_labels = readdata()
    cross_validation(all_matrix, all_labels, 'DNN', event_num, seed, CV)

In [26]:
main()

_____cross_validation_____
________DNN_________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Inputlayer (InputLayer)      (None, 3390)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               1736192   
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 512)               2048      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 256)               1024      
_________________________________________________________________
dropout_5 (Dropout)         

Epoch 1/100
12708/12708 [==============================] - 9s 678us/sample - loss: 0.6721 - acc: 0.7141 - val_loss: 0.4443 - val_acc: 0.8003
Epoch 2/100
12708/12708 [==============================] - 8s 637us/sample - loss: 0.4094 - acc: 0.8178 - val_loss: 0.4049 - val_acc: 0.8187
Epoch 3/100
12708/12708 [==============================] - 8s 642us/sample - loss: 0.3574 - acc: 0.8362 - val_loss: 0.3813 - val_acc: 0.8201
Epoch 4/100
12708/12708 [==============================] - 8s 640us/sample - loss: 0.3206 - acc: 0.8469 - val_loss: 0.4031 - val_acc: 0.8095
Epoch 5/100
12708/12708 [==============================] - 8s 656us/sample - loss: 0.2967 - acc: 0.8533 - val_loss: 0.3765 - val_acc: 0.8109
Epoch 6/100
12708/12708 [==============================] - 8s 640us/sample - loss: 0.2845 - acc: 0.8556 - val_loss: 0.3727 - val_acc: 0.8137
Epoch 7/100
12708/12708 [==============================] - 8s 645us/sample - loss: 0.2759 - acc: 0.8572 - val_loss: 0.3722 - val_acc: 0.8173
Epoch 8/100
1

12708/12708 [==============================] - 8s 644us/sample - loss: 0.4104 - acc: 0.8207 - val_loss: 0.4711 - val_acc: 0.7790
Epoch 3/100
12708/12708 [==============================] - 8s 666us/sample - loss: 0.3480 - acc: 0.8456 - val_loss: 0.4383 - val_acc: 0.7833
Epoch 4/100
12708/12708 [==============================] - 8s 650us/sample - loss: 0.3155 - acc: 0.8494 - val_loss: 0.4367 - val_acc: 0.7875
Epoch 5/100
12708/12708 [==============================] - 8s 644us/sample - loss: 0.2966 - acc: 0.8557 - val_loss: 0.4115 - val_acc: 0.7918
Epoch 6/100
12708/12708 [==============================] - 8s 652us/sample - loss: 0.2805 - acc: 0.8613 - val_loss: 0.4158 - val_acc: 0.7911
Epoch 7/100
12708/12708 [==============================] - 8s 651us/sample - loss: 0.2703 - acc: 0.8590 - val_loss: 0.4116 - val_acc: 0.7904
Epoch 8/100
12708/12708 [==============================] - 8s 653us/sample - loss: 0.2635 - acc: 0.8609 - val_loss: 0.3995 - val_acc: 0.7925
Epoch 9/100
12708/12708 [

Epoch 1/100
12708/12708 [==============================] - 9s 702us/sample - loss: 0.6487 - acc: 0.7194 - val_loss: 0.4694 - val_acc: 0.7805
Epoch 2/100
12708/12708 [==============================] - 8s 654us/sample - loss: 0.4039 - acc: 0.8234 - val_loss: 0.4486 - val_acc: 0.7960
Epoch 3/100
12708/12708 [==============================] - 8s 651us/sample - loss: 0.3445 - acc: 0.8417 - val_loss: 0.4158 - val_acc: 0.8031
Epoch 4/100
12708/12708 [==============================] - 8s 646us/sample - loss: 0.3172 - acc: 0.8493 - val_loss: 0.4159 - val_acc: 0.8067
Epoch 5/100
12708/12708 [==============================] - 8s 649us/sample - loss: 0.2967 - acc: 0.8528 - val_loss: 0.4150 - val_acc: 0.8017
Epoch 6/100
12708/12708 [==============================] - 8s 654us/sample - loss: 0.2780 - acc: 0.8604 - val_loss: 0.4289 - val_acc: 0.7939
Epoch 7/100
12708/12708 [==============================] - 8s 653us/sample - loss: 0.2702 - acc: 0.8610 - val_loss: 0.4118 - val_acc: 0.8045
Epoch 8/100
1

Epoch 1/100
12708/12708 [==============================] - 9s 707us/sample - loss: 0.6755 - acc: 0.7101 - val_loss: 0.4453 - val_acc: 0.7989
Epoch 2/100
12708/12708 [==============================] - 9s 669us/sample - loss: 0.4134 - acc: 0.8191 - val_loss: 0.4047 - val_acc: 0.8067
Epoch 3/100
12708/12708 [==============================] - 8s 657us/sample - loss: 0.3438 - acc: 0.8437 - val_loss: 0.3817 - val_acc: 0.8144
Epoch 4/100
12708/12708 [==============================] - 8s 663us/sample - loss: 0.3279 - acc: 0.8458 - val_loss: 0.3783 - val_acc: 0.8229
Epoch 5/100
12708/12708 [==============================] - 8s 654us/sample - loss: 0.3035 - acc: 0.8520 - val_loss: 0.3705 - val_acc: 0.8364
Epoch 6/100
12708/12708 [==============================] - 8s 663us/sample - loss: 0.2895 - acc: 0.8557 - val_loss: 0.3618 - val_acc: 0.8137
Epoch 7/100
12708/12708 [==============================] - 9s 689us/sample - loss: 0.2780 - acc: 0.8541 - val_loss: 0.3562 - val_acc: 0.8244
Epoch 8/100
1